In [ ]:
## DOWNLOAD DATA FROM CRAIG LIST
import sys
import requests
import csv

from lxml import etree
from io import StringIO as StringIO, BytesIO

page_link = 'https://annarbor.craigslist.org/search/cta'
pages_links_list =[]

while page_link is not None:
    page = requests.get(url = page_link)
    parser = etree.HTMLParser()
    tree = etree.parse(StringIO(page.text), parser)
    links_on_page = tree.xpath("//a[@class='result-title hdrlnk']/@href")
    pages_links_list.extend(links_on_page)  
    next_page_link = tree.xpath("//a[@title='next page']/@href")
    if len(next_page_link) > 0:
        page_link = 'https://annarbor.craigslist.org' + next_page_link[0]
    else:
        page_link = None
        
def get_first_element(elements):
    if len(elements) > 0:
        return elements[0].strip()
    else:
        return ''

def get_empty_page_info():
    empty_page_info = dict.fromkeys(['url'
        , 'price'
        , 'year'
        , 'make'
        , 'model'
        , 'cylinders'
        , 'drive'
        , 'fuel'
        , 'odometer'
        , 'paint color'
        , 'transmission'
        , 'posted'
        , 'updated'])
    return empty_page_info

pages = []
for page_link in pages_links_list:
    page_info = get_empty_page_info()
    page_info['url'] = page_link
    page = requests.get(url = page_link)
    parser = etree.HTMLParser()
    tree = etree.parse(StringIO(page.text), parser)
    page_info['price'] = (get_first_element(tree.xpath("//span[@class='price']/text()"))
                            .replace('$', ''))
    span_elements = tree.xpath("//div[@class='mapAndAttrs']/p/span")
    fields_list = list(page_info.keys())
    if len(span_elements) > 0:
        year_make_model = span_elements[0].xpath("b/text()")[0].split(' ')
        page_info['year'] = year_make_model[0]
        if len(year_make_model) > 2:
            page_info['make'] = year_make_model[1]
            page_info['model'] = year_make_model[2]
        else:
            page_info['make'] = year_make_model[1]
            page_info['model'] = ''
        for span_element in span_elements:
            field = get_first_element(span_element.xpath("text()")).replace(':', '')
            if field in fields_list:
                page_info[field] = get_first_element(span_element.xpath("b/text()")) 
        page_info['posted'] = get_first_element(tree.xpath("//time[@class='date timeago']/@datetime"))       
    page_info_res = {}
    for k, v in page_info.items():
        page_info_res[k] = '' if v is None else v.lower()
    pages.append(page_info_res)
    
with open('annarbor.csv', 'w') as csvfile:
    fieldnames = list(get_empty_page_info().keys())
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for page in pages:
        writer.writerow(page)